In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'QuadruppedWalk-v1'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

robot lib not fouund
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-06-16 19:01:50,404	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-16_19-01-50_11590/logs.
2019-06-16 19:01:50,517	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:26930 to respond...
2019-06-16 19:01:50,665	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:27665 to respond...
2019-06-16 19:01:50,674	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-06-16 19:01:50,733	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-16_19-01-50_11590/logs.
2019-06-16 19:01:50,736	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.105',
 'redis_address': '192.168.3.105:26930',
 'object_store_address': '/tmp/ray/session_2019-06-16_19-01-50_11590/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-06-16_19-01-50_11590/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks
import games_configurations

#a2c_config = games_configurations.bipedalwalker_config
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
a2c_config = games_configurations.quadrupped_config
#a2c_config = a2c_games_configurations.mountain_car_cont_config
agent = A2CAgent(sess,'walker', obs_space, False, action_space, a2c_config)
agent.restore('nn/walkerQuadruppedWalk-v1')
#agent.train()
#agent.save('nn/latest')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


2019-06-16 19:01:51,803	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from nn/walkerQuadruppedWalk-v1


In [ ]:
ray.shutdown()

In [ ]:
import numpy as np
#agent.save('nn/latest_q')
#agent.restore('nn/carRoboschoolAnt-v1')
#agent.restore('nn/rob_20QuadruppedWalk-v1')
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')
env = env_configurations.configurations[env_name]['ENV_CREATOR']()
def evaluate(env,t_max=5000):
    rewards = []
    #env._max_episode_steps = 5000
    env.batch_mode = True
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env.close()




WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
reset
[[-0.4774567  -0.5433008  -0.77245355 -0.02087444  0.3023395  -0.7729136
   0.01014106 -0.17652974 -0.95033145  0.64272565 -0.47361982 -0.5937056 ]]
[[-0.43920457 -0.6943674  -0.8902069   0.09806534  0.17407033 -0.8480541
   0.09562773 -0.30891022 -0.9916935   0.80495584 -0.6148219  -0.76456296]]
[[ 0.14425942 -0.06748626 -0.9785811  -0.6970147   0.23356888 -0.7315477
   0.10940602 -0.86325073 -0.9159468   0.7004858  -0.7174759  -0.4734763 ]]
[[ 0.4140496  -0.19420783 -0.93603957 -0.4908632  -0.07362204 -0.84679645
  -0.47552413 -0.81077045 -0.89779663  0.84263456 -0.9579602  -0.6743874 ]]
[[ 0.32962316  0.44882146 -0.9504932  -0.8082561  -0.12951005 -1.
  -0.8749123  -0.9433068  -1.          0.07751334 -0.58192474 -0.5448904 ]]
[[ 0.5531131   0.3010921  -0.8898682  -0.40677142 -0.4847

[[-0.77021444 -0.21545064 -0.789209    0.39953256  0.525541   -0.62054634
   0.6674631   0.5552239  -0.6395263   0.3768528  -0.18378857 -0.74480176]]
[[-0.14054771 -0.46971574 -0.893291   -0.26200187  0.8662827  -0.82331836
   0.96903116  0.27040252 -0.79045033  0.25436908 -0.24150033 -0.88601905]]
[[-0.15076064 -0.3958447  -1.         -0.4666721   0.94470054 -0.94783616
   0.9465654  -0.11717656 -0.83869874  0.7716742  -0.49490353 -0.86149895]]
[[-0.26936242 -0.31633753 -0.8493256  -0.35799715  0.4877559  -0.89762396
   0.70041364 -0.53437585 -0.9196679   0.5303587  -0.49067068 -0.7567577 ]]
[[ 0.0637847  -0.20653254 -0.92101365 -0.6280798   0.5517355  -1.
   0.17577288 -0.84467167 -0.84254897  0.77930635 -0.8153268  -0.75063044]]
[[ 0.2618065   0.26455635 -0.9440615  -0.8841087   0.22109435 -0.93352574
  -0.5980242  -1.         -0.901731    0.6751816  -0.89220923 -0.6748952 ]]
[[ 0.5351822   0.06937334 -0.7911693  -0.7964723   0.17702897 -0.72650164
  -0.81348413 -0.90885144 -0.83183

[[-0.09029417 -0.22812639 -0.90285695 -0.32348016  0.6165301  -0.8395711
   0.9107388  -0.07729485 -0.9021324   0.740674   -0.34216002 -0.8887654 ]]
[[ 0.34810942 -0.17455816 -0.9986412  -1.          0.60871494 -0.8753259
   0.5693145  -0.84182835 -0.7777418   1.         -0.8201143  -0.94076407]]
[[ 0.28563178 -0.01113391 -0.85469896 -0.71747553  0.40240416 -0.9565885
  -0.31866783 -0.9038677  -0.95266205  0.6886412  -0.82994974 -0.48574933]]
[[ 0.61605096 -0.06012809 -0.8932965  -0.54625106  0.21902546 -1.
  -0.65451056 -0.8213034  -0.8691989   0.17912613 -0.5620517  -0.5262674 ]]
[[ 6.6839415e-01  9.2703313e-02 -8.2201368e-01 -4.9081188e-01
  -5.4163486e-04 -8.1616640e-01 -7.2022951e-01 -8.2249761e-01
  -8.8695788e-01 -4.6072531e-01 -3.4044534e-01 -8.4360087e-01]]
[[ 0.86373043  0.2918765  -0.8119571   0.51012814 -0.4849003  -0.08799332
  -0.8242928  -0.8747247  -1.         -0.48375884 -0.32107234 -0.8360941 ]]
[[ 0.36715078  0.18340811 -0.74020934  0.47930428 -0.7400986  -0.6552616


[[ 0.11193191 -0.29149917 -0.97193223 -0.4798361   0.73085886 -0.88662726
   0.96511275 -0.59047836 -0.80038494  0.671195   -0.43993756 -0.83047014]]
[[-0.58960664 -0.825356   -0.7814409  -0.4102388   0.4749699  -0.94624525
   0.75310874 -0.55748326 -0.8626775   0.80961454 -0.66595626 -0.8041429 ]]
[[ 0.33984315 -0.17241308 -0.90819573 -0.45954552  0.32593268 -0.8782948
   0.24077328 -0.87065905 -0.90178865  1.         -0.8801901  -0.67866176]]
[[ 0.5074219   0.31345338 -0.95880896 -0.6722528   0.3435365  -0.8658367
  -0.6545421  -0.94120806 -0.8888216   0.53731555 -0.71505606 -0.6234969 ]]
[[ 0.81037235  0.38992065 -0.92172086 -0.6298794   0.08485143 -0.91546714
  -0.9944767  -0.9827035  -0.90665716  0.18735188 -0.6986639  -0.6766534 ]]
[[ 0.84737575  0.6286541  -0.95141286 -0.2892207  -0.14496394 -1.
  -1.         -0.8222827  -0.75760084 -0.26762015 -0.3128805  -0.68752867]]
[[ 0.7799287   0.12738866 -0.7500232  -0.22733125 -0.63406146 -0.36586097
  -0.97467124 -0.533211   -0.9779448

[[ 0.43070117  0.18353431 -0.8546031  -0.6990551   0.07971843 -0.81615347
  -0.86217624 -0.7884393  -0.99643075  0.5198986  -0.77512145 -0.5078962 ]]
[[ 0.5292795   0.6193981  -0.7271594  -0.8669317  -0.18866053 -0.82260406
  -0.91388595 -0.96336573 -0.89728624 -0.51714796 -0.3097757  -0.825968  ]]
[[ 0.8115016   0.71092397 -0.9323413  -0.55475026 -0.03636824 -0.69382626
  -1.         -0.82070637 -0.8051065  -0.9415984   0.01484445 -1.        ]]
[[ 0.83382094  0.42407107 -0.9528913  -0.11214472 -0.35888213 -0.5170184
  -0.8867363  -1.         -0.82671124 -0.47778895  0.26916283 -1.        ]]
[[ 0.76344    -0.1112784  -0.79102135  0.6789917  -0.8625027  -0.59272915
  -0.94709927 -0.10175605 -0.67509884 -0.36575204  0.2155186  -0.9390408 ]]
[[ 0.7071442   0.08886263 -0.962142    0.8109939  -0.90409404 -0.85990685
  -1.          0.43518028 -0.7664655  -0.4174981   0.0148332  -0.95194423]]
[[ 0.5847362  -0.06933191 -1.          1.         -1.         -0.52201897
  -0.88504815  0.81609225 -

...

[[-0.22770342 -0.22522315 -0.66979986  1.         -0.27979395 -0.42709967
  -0.40548286  0.88139915 -0.7460632  -0.12361326  0.0696361  -0.7550133 ]]
[[-0.72813916 -0.4428724  -0.8897071   0.7801534   0.0193373  -0.9379777
   0.4279677   0.4706043  -0.91746974  0.49559936 -0.23831725 -0.85671586]]
[[-0.71617496 -0.35884777 -0.88343775  0.32968217  0.36819738 -0.8051881
   0.6164345   0.625216   -0.7810575   0.13141562 -0.26470962 -1.        ]]
[[-0.33046588 -0.20427345 -0.93105835 -0.09547245  0.76121736 -0.88808477
   0.8340229  -0.0020898  -0.57304096  0.31133452 -0.66208076 -1.        ]]
[[-0.41425014 -0.5598762  -0.7387247  -0.26916352  0.8211372  -0.6583916
   0.7021296  -0.65096784 -0.68235195  0.8128361  -0.503064   -0.69634557]]
[[-0.25568095 -0.23890509 -0.89369017 -0.47483063  0.7521136  -0.8309892
   0.529383   -0.702389   -0.89989454  0.7085509  -0.57638466 -0.55190057]]
[[-0.0371829  -0.45182484 -0.9204226  -0.5880804   0.08658876 -0.84785736
   0.4483956  -0.8026335  -0.7

[[-0.12867431 -0.3266319  -0.8532907  -0.6063635   0.8239874  -0.9744344
   0.813526   -0.62347215 -0.8214661   0.6672128  -0.6798629  -0.6539444 ]]
[[-0.1344943  -0.17175001 -0.9939238  -0.5380511   0.48203522 -0.79377097
   0.31218043 -0.83243775 -0.9280009   0.9412507  -0.75698054 -0.8664943 ]]
[[-0.0962683   0.01657805 -1.         -0.7781954   0.46083748 -0.7404702
  -0.22678773 -0.8515491  -0.9465699   0.8267731  -0.8808332  -0.78150904]]
[[ 0.21125205  0.12084161 -0.96170014 -0.8191528   0.15280817 -0.78283805
  -0.8055058  -0.8964873  -0.9534461   0.11644986 -0.77625465 -0.6720712 ]]
[[ 0.6849933   0.6712941  -0.8349704  -0.5230231  -0.18680966 -0.76692444
  -0.8630314  -0.850168   -0.9387237  -0.09852906 -0.45830247 -0.82464355]]
[[ 0.3812325   0.40308988 -0.8920113  -0.34049225 -0.80064017 -1.
  -1.         -0.9836231  -0.62247133 -0.5678729  -0.00908288 -0.7191417 ]]
[[ 1.          0.07372542 -0.8312361   0.447059   -0.86957383 -0.59616524
  -0.8754011  -0.14982267 -0.7979966

[[-0.2507112  -0.55027556 -1.         -0.82443416  0.4225196  -0.7310959
   0.04236436 -0.36507708 -0.5705582   0.7543718  -0.28364214 -1.        ]]
[[-0.24497956 -0.47989082 -0.9650266  -0.6679508   0.7037154  -0.934625
   0.8012558  -0.59958214 -0.91200805  0.77580297 -0.7073322  -0.7844229 ]]
[[-0.233649   -0.5905197  -0.91518646 -0.7417767   0.51731735 -0.84074336
   0.7306841  -0.7278294  -0.79223317  0.95269895 -0.7391073  -0.67966056]]
[[ 0.45762342  0.23668543 -0.97544914 -0.8753399   0.21735582 -0.9077403
  -0.6222046  -0.9831079  -0.96089584  0.77352333 -0.8962821  -0.539608  ]]
[[ 0.8679349   0.38900164 -0.8275981  -0.8706672  -0.23286957 -0.97500193
  -0.86369246 -0.997095   -0.9454156   0.11626771 -0.79353654 -0.70108205]]
[[ 0.9131104   0.82077086 -0.9246905  -0.6712148  -0.14644654 -0.6385387
  -0.90328157 -0.9296129  -0.7556668  -0.32140735 -0.57369995 -0.77343905]]
[[ 1.          0.62515044 -0.69648874  0.04120916 -0.7406734  -0.96234334
  -0.995236   -0.8055304  -0.74

[[ 0.4346205  -0.20446502 -0.9576829  -0.6215181   0.686303   -0.9085856
   0.5602831  -1.         -0.8855025   0.97644615 -0.82558286 -0.73754346]]
[[ 0.59298193  0.11693344 -0.9160494  -0.79755807  0.10658896 -0.882916
  -0.25718778 -0.88066363 -0.9497199   0.85183626 -0.88288355 -0.6076285 ]]
[[ 0.673844    0.09180798 -0.98550975 -0.94673246 -0.06007498 -0.81237054
  -0.59328276 -0.8521678  -0.93930215  0.2555668  -0.5714173  -0.5895989 ]]
[[ 0.6505946  -0.07747668 -0.93599695 -0.54530114 -0.06811472 -0.83795685
  -0.8256294  -0.8407195  -0.98798084 -0.26785302 -0.6308354  -0.6601579 ]]
[[ 0.8599538   0.64417934 -0.78446627  0.00664195 -0.83540845 -0.7774104
  -0.94037324 -0.947832   -0.9732411  -0.40880087  0.07825092 -0.81561196]]
[[ 0.7047812  -0.37535977 -0.47420132  0.4277184  -0.9637222  -0.69591737
  -0.99942803 -0.02450683 -0.83495617 -0.65672934  0.8305695  -0.99549544]]
[[ 0.06632823 -0.36949193 -0.69962615  0.44774175 -0.75875616 -0.73908615
  -0.8435345   0.5054276  -0.6

[[-0.44900036 -0.16465369 -0.6346242   0.9443065  -0.7583623  -0.80560464
  -0.06271824  0.5093812  -0.91231245  0.32148063  0.04453289 -0.7333842 ]]
[[-0.40858948  0.00480017 -0.7189521   0.9376222  -0.39750445 -1.
  -0.11938056  0.6188712  -0.8659495  -0.03404541 -0.4805521  -0.8282307 ]]
[[-0.9429567  -0.5351708  -0.85252666  0.6301745  -0.11720796 -0.55995864
  -0.09681898  0.7212414  -0.9093132   0.23767497 -0.3403518  -0.8385316 ]]
[[-0.58466905 -0.38024372 -0.8972658   0.6430517   0.07040643 -0.6194551
   0.38975117  0.67696494 -0.5624726   0.03565596 -0.13677943 -0.890343  ]]
[[-0.4130911  -0.26727778 -0.7745054   0.51705253  0.68228334 -0.9653586
   0.9125655   0.5259259  -0.9184114   0.35333204  0.00589886 -0.9760337 ]]
[[-0.5525049  -0.4887883  -0.7957199  -0.00253283  0.7356092  -0.48699003
   0.5260912   0.17625529 -0.7651553   0.703082   -0.29975954 -0.6195215 ]]
[[-0.2033726  -0.17567027 -0.9621051  -0.2580534   0.78988564 -0.878447
   0.5291945   0.22255965 -0.772635   

In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()